---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms

states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}


In [80]:
'''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''

import re
def get_list_of_university_towns():
    
    def limpiar(str_):
        str_ = re.sub(r'\[.*\]',"",str_)
        str_ = str_.replace("\n","")
        str_ = str_.strip()
        return str_
        
        
    df = pd.read_table('university_towns.txt', header = None )
    df.columns = ['RegionName']
    df['State'] =  df['RegionName'][df['RegionName'].str.endswith('[edit]')]          # endswith ：判断字符串是否以指定后缀结尾
    df['State'] =  df['State'].fillna(method='pad')                             # 用前一个数据代替NaN（ method = ’pad‘）
    df['State'] = df['State'].str.split('\[').str[0]                     # remove '[edit]' from df['State']
    df = df[ ~ df['RegionName'].str.contains('edit') ]              # remove rows contain '[edit]'
    df['RegionName'] = df['RegionName'].str.split('\(').str[0]             # remove ()后面的内容
    
    df['State'] = [limpiar(str) for str in df['State']]
    df['RegionName'] = [limpiar(str) for str in df['RegionName']]
    
    df = df.set_index('State').reset_index()
    
    return df
get_list_of_university_towns()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
5,Alabama,Troy
6,Alabama,Tuscaloosa
7,Alabama,Tuskegee
8,Alaska,Fairbanks
9,Arizona,Flagstaff


In [187]:
'''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''

def get_recession_start():
    # clean data
    df_GDP = pd.read_excel('gdplev.xls', skiprows = 8, names = ['c1','c2','c3','c4','c5','c6','c7','c8'])
    df_GDP = df_GDP[['c5', 'c7']]
    df_GDP.columns = ['Quarter', 'GDP']
    df_GDP = df_GDP[df_GDP.Quarter.str[:4] > '1999'] 

#     diff = np.array(df_GDP['GDP'].diff().fillna(0))
#     list1 = (diff < 0) + 0
#     list2 = np.concatenate([list1[1:],[0]])
#     index = np.where((list1*list2)>0)
#     quarter  = np.array(df_GDP['Quarter'][index[0]])[0]

    recession_start = []
    for i in range(len(df_GDP) - 4):
        if ((df_GDP.iloc[i][1] > df_GDP.iloc[i+1][1]) & (df_GDP.iloc[i+1][1] > df_GDP.iloc[i+2][1])):
            recession_start.append(df_GDP.iloc[i][0])

    ans = recession_start[0]
    return ans

get_recession_start()

'2008q2'

In [182]:
 '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
       

def get_recession_end():
    df_GDP = pd.read_excel('gdplev.xls', skiprows = 8, names = ['c1','c2','c3','c4','c5','c6','c7','c8'])
    df_GDP = df_GDP[['c5', 'c7']]
    df_GDP.columns = ['Quarter', 'GDP']
    df_GDP = df_GDP[df_GDP.Quarter.str[:4] > '1999'] 

    
    recession_end = []
    for i in range(len(df_GDP) - 4):
        if ((df_GDP.iloc[i][1] > df_GDP.iloc[i+1][1]) & (df_GDP.iloc[i+1][1] > df_GDP.iloc[i+2][1]) & (df_GDP.iloc[i+2][1] < df_GDP.iloc[i+3][1]) & (df_GDP.iloc[i+3][1] < df_GDP.iloc[i+4][1])):
            recession_end.append([df_GDP.iloc[i][0],df_GDP.iloc[i+1][0],df_GDP.iloc[i+2][0],df_GDP.iloc[i+3][0],df_GDP.iloc[i+4][0]])

    ans = recession_end[0][4]
    return ans

get_recession_end()

'2009q4'

In [178]:
 '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
def get_recession_bottom():
    df = pd.read_excel('gdplev.xls', skiprows = 8, names = ['c1','c2','c3','c4','c5','c6','c7','c8'])
    df = df[['c5', 'c6']]
    df.columns = ['Quarter', 'GDP']
    df = df[df.Quarter.str[:4] > '1999'] 
    
    recession_end = []
    for i in range(len(df) - 4):
        if ((df.iloc[i][1] > df.iloc[i+1][1]) & (df.iloc[i+1][1] > df.iloc[i+2][1]) & (df.iloc[i+2][1] < df.iloc[i+3][1]) & (df.iloc[i+3][1] < df.iloc[i+4][1])):
            recession_end.append([df.iloc[i][0],df.iloc[i+1][0],df.iloc[i+2][0],df.iloc[i+3][0],df.iloc[i+4][0]])

    ans = recession_end[0][2]
    
    return ans

get_recession_bottom()

'2009q2'

In [81]:
  '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
def convert_housing_data_to_quarters():
    df_h = pd.read_csv('City_Zhvi_AllHomes.csv')
    df_h = df_h.drop(['RegionID', 'Metro','CountyName','SizeRank'],axis=1)
    
    
    states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}
    State_full=[states[key] for key in df_h['State']]
    df_h['State'] = State_full
    
    df_h = df_h.set_index(['State', 'RegionName'])
    mycolumns = df_h.columns[df_h.columns.str.contains('20')]
    df_h = df_h[mycolumns]
      
    for i in range(2000,2016):
        df_h[(str(i)+'q1')] = df_h[[str(i)+'-0'+str(l) for l in range(1,4)]].mean(axis=1)
        df_h[(str(i)+'q2')] = df_h[[str(i)+'-0'+str(l) for l in range(4,7)]].mean(axis=1)
        df_h[(str(i)+'q3')] = df_h[[str(i)+'-0'+str(l) for l in range(7,10)]].mean(axis=1)
        df_h[(str(i)+'q4')] = df_h[[str(i)+'-'+str(l) for l in range(10,13)]].mean(axis=1)
    
    df_h[('2016q1')] = df_h[[str(i)+'-0'+str(l) for l in range(1,4)]].mean(axis=1)  
    df_h[('2016q2')] = df_h[[str(i)+'-0'+str(l) for l in range(4,7)]].mean(axis=1)    
    df_h[('2016q3')] = df_h[[str(i)+'-0'+str(l) for l in range(7,10)]].mean(axis=1)
    
    df_h = df_h.drop(mycolumns,axis=1)   
    
    return  df_h
convert_housing_data_to_quarters()

,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,515466.666667,522800.000000,528066.666667,532266.666667,540800.000000,557200.000000,572833.333333,532266.666667,540800.000000,557200.000000
California,Los Angeles,207066.666667,214466.666667,220966.666667,226166.666667,233000.000000,239100.000000,245066.666667,253033.333333,261966.666667,272700.000000,...,498033.333333,509066.666667,518866.666667,528800.000000,538166.666667,547266.666667,557733.333333,528800.000000,538166.666667,547266.666667
Illinois,Chicago,138400.000000,143633.333333,147866.666667,152133.333333,156933.333333,161800.000000,166400.000000,170433.333333,175500.000000,177566.666667,...,192633.333333,195766.666667,201266.666667,201066.666667,206033.333333,208300.000000,207900.000000,201066.666667,206033.333333,208300.000000
Pennsylvania,Philadelphia,53000.000000,53633.333333,54133.333333,54700.000000,55333.333333,55533.333333,56266.666667,57533.333333,59133.333333,60733.333333,...,113733.333333,115300.000000,115666.666667,116200.000000,117966.666667,121233.333333,122200.000000,116200.000000,117966.666667,121233.333333
Arizona,Phoenix,111833.333333,114366.666667,116000.000000,117400.000000,119600.000000,121566.666667,122700.000000,124300.000000,126533.333333,128366.666667,...,164266.666667,165366.666667,168500.000000,171533.333333,174166.666667,179066.666667,183833.333333,171533.333333,174166.666667,179066.666667


In [188]:
 '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
def run_ttest():
    
    unitowns = get_list_of_university_towns()
    bottom = get_recession_bottom()
    start = get_recession_start()
    hdata = convert_housing_data_to_quarters()
    bstart = hdata.columns[hdata.columns.get_loc(start) - 1]
    
    hdata['ratio'] =  hdata[bstart] - hdata[bottom]
    hdata = hdata[[bottom, bstart, 'ratio']]
    hdata = hdata.reset_index()
    
    unitowns_hdata = pd.merge(hdata,unitowns,how='inner',on=['State','RegionName'])
    unitowns_hdata['uni'] = True
    hdata2 = pd.merge(hdata, unitowns_hdata, how='outer', on=['State','RegionName',bottom, bstart, 'ratio'])
    hdata2['uni'] = hdata2['uni'].fillna(False)
    
    ut = hdata2[hdata2['uni'] == True]
    nut = hdata2[hdata2['uni'] == False]
    
    t,p = ttest_ind(ut['ratio'].dropna(), nut['ratio'].dropna())
    
    different = True if p<0.01 else False
    better = "university town" if ut['ratio'].mean() < nut['ratio'].mean() else "non-university town"
    
    return (different, p, better)

#     def get_session(row_index):
#         diff = np.array(df_h.iloc[row_index,2:].diff())
#         a = (diff < 0) + 0
        
#         start_tmp = []
#         for i in range(len(a)-1):
#             if a[i] == 1 and a[i+1] == 1:
#                 start_tmp.append(i)
#             else:
#                 i = i + 1
#         end_temp = []
#         session = []
#         for index in start_tmp:
#             for i in range(index, len(a)-2):
#                 if a[i]==0 and a[i+1]==0:
#                     end_temp.append(i+1)
#                     session.append((index,i+1))
#                     break
#                 else:
#                     i = i + 1
#         return session
    
#     nsession = []
#     for region in range(df_h.shape[0]):
#         nsession.append(len(get_session(region)))
        

    
#     print(nsession)
run_ttest()

,State,RegionName,2009q2,2008q1,ratio
0,New York,New York,4.658333e+05,5.085000e+05,42666.666667
1,California,Los Angeles,4.139000e+05,5.353000e+05,121400.000000
2,Illinois,Chicago,2.197000e+05,2.437333e+05,24033.333333
3,Pennsylvania,Philadelphia,1.161667e+05,1.195667e+05,3400.000000
4,Arizona,Phoenix,1.682333e+05,2.186333e+05,50400.000000
5,Nevada,Las Vegas,1.643333e+05,2.509333e+05,86600.000000
6,California,San Diego,3.895000e+05,4.619667e+05,72466.666667
7,Texas,Dallas,1.051000e+05,1.199000e+05,14800.000000
8,California,San Jose,5.303000e+05,6.430667e+05,112766.666667
9,Florida,Jacksonville,1.408333e+05,1.651667e+05,24333.333333
